# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    #1 TODO: Implement Function
    
    #Count frequency of words
    counts = Counter(text)
    #Sort words
    vocab = sorted(counts, key=counts.get, reverse=True)
    #Create a tuple of words-to-integers
    vocab_to_int = {word: i for i, word in enumerate(vocab, 0)}
    #Create a tuple of integers-to-words
    int_to_vocab = dict(enumerate(vocab, 0))
    #Return both dictionaries
    return vocab_to_int, int_to_vocab

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Create a dictionary which replaces punctuation with tokens.
    Then return the tokenized dictionary: The punctuation is the key, the punctuation terms are token values.
    """
    #2 TODO: Implement Function

    table = {'.': '|period|',
             ',': '|comma|',
             '"': '|quotation_mark|',
             ';': '|semicolon|',
             '!': '|exclamation_mark|',
             '?': '|question_mark|',
             '(': '|left_parentheses|',
             ')': '|right_parentheses|',
             '--': '|dash|',
             '\n': '|return|'}
    return table


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1


/home/michael/anaconda3/envs/tv-script/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    #3 TODO: Implement Function
    #Create placeholders for input, targets, and learning rate
    #Then return tuple containing inputs, targets, and learning rate
    
    input        = tf.placeholder(tf.int32, shape=(None, None), name='input')  #MPCHANGED inputs to input (both)
    targets       = tf.placeholder(tf.int32, shape=(None, None), name='targets')
    learning_rate = tf.placeholder(tf.float32, shape=(None), name='learning_rate')
    return (input, targets, learning_rate)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    #4 TODO: Implement Function
    lstm_layers = 2
    # Create a LSTM cell; Begin making RNN
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_size)
    #Apply dropout layer
    dropout = tf.contrib.rnn.DropoutWrapper(lstm_cell, 0.5)
    # Create RNN Cell using size 
    rnn_cell = tf.contrib.rnn.MultiRNNCell([dropout] * lstm_layers)
    #Initialize rnn cell using fp zeros of single precision
    initial_state = rnn_cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state')
    # return rnn_cell and initial_state
    return rnn_cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:

def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    #5 TODO: Implement Function
    
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed
   
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)http://localhost:8888/notebooks/dlnd_tv_script_generation-Copy2.ipynb#
    """
    #6 TODO: Implement Function
    
    # Create RNN cell
    outputs, final_state = tf.nn.dynamic_rnn(cell,inputs,dtype=tf.float32)
    # Apply Name as 'final_state'
    final_state = tf.identity(final_state, name="final_state")
    #Return outputs and final_state
    return outputs, final_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    #7 MPTESTING:  TODO: Implement Function

    #Get embedded inputs
    embed = get_embed(input_data, vocab_size, embed_dim=rnn_size)
    #Get rnn outputs
    outputs, final_state = build_rnn(cell, embed)
    #Apply fully connected layer
    logits = tf.contrib.layers.fully_connected(
        inputs = outputs,
        num_outputs = vocab_size,
        activation_fn=None,
        weights_initializer=tf.truncated_normal_initializer(stddev= 0.1),
        biases_initializer=tf.zeros_initializer())
    #Return logits and final_state
    return logits, final_state

    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    #8 TODO: Implement Function
    
    n_batches = len(int_text)//(batch_size * seq_length)
    tr_int_text = int_text[:batch_size * seq_length * n_batches]
    tr_output_int_text = tr_int_text[1:] + tr_int_text[0:1]
    batches = np.zeros((n_batches, 2, batch_size, seq_length))
    for i in range(n_batches):
        for j in range(batch_size):
            cursor = n_batches*j+i
            batches[i][0][j] = tr_int_text[seq_length*cursor   : seq_length * (cursor + 1)]
            batches[i][1][j] = tr_output_int_text[seq_length*cursor : seq_length * (cursor + 1)]

    return batches
 
    
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [14]:
# Number of Epochs
num_epochs = 200
# Batch Size
batch_size = 128 
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 5
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 5

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/107   train_loss = 8.824
Epoch   0 Batch    5/107   train_loss = 8.740
Epoch   0 Batch   10/107   train_loss = 8.386
Epoch   0 Batch   15/107   train_loss = 7.823
Epoch   0 Batch   20/107   train_loss = 7.243
Epoch   0 Batch   25/107   train_loss = 6.778
Epoch   0 Batch   30/107   train_loss = 6.369
Epoch   0 Batch   35/107   train_loss = 6.406
Epoch   0 Batch   40/107   train_loss = 6.571
Epoch   0 Batch   45/107   train_loss = 6.305
Epoch   0 Batch   50/107   train_loss = 5.989
Epoch   0 Batch   55/107   train_loss = 6.205
Epoch   0 Batch   60/107   train_loss = 6.120
Epoch   0 Batch   65/107   train_loss = 6.171
Epoch   0 Batch   70/107   train_loss = 6.111
Epoch   0 Batch   75/107   train_loss = 5.857
Epoch   0 Batch   80/107   train_loss = 5.971
Epoch   0 Batch   85/107   train_loss = 5.703
Epoch   0 Batch   90/107   train_loss = 5.882
Epoch   0 Batch   95/107   train_loss = 5.682
Epoch   0 Batch  100/107   train_loss = 5.673
Epoch   0 Batch  105/107   train_l

Epoch   8 Batch   39/107   train_loss = 4.494
Epoch   8 Batch   44/107   train_loss = 4.427
Epoch   8 Batch   49/107   train_loss = 4.545
Epoch   8 Batch   54/107   train_loss = 4.496
Epoch   8 Batch   59/107   train_loss = 4.520
Epoch   8 Batch   64/107   train_loss = 4.505
Epoch   8 Batch   69/107   train_loss = 4.511
Epoch   8 Batch   74/107   train_loss = 4.522
Epoch   8 Batch   79/107   train_loss = 4.506
Epoch   8 Batch   84/107   train_loss = 4.567
Epoch   8 Batch   89/107   train_loss = 4.744
Epoch   8 Batch   94/107   train_loss = 4.464
Epoch   8 Batch   99/107   train_loss = 4.583
Epoch   8 Batch  104/107   train_loss = 4.531
Epoch   9 Batch    2/107   train_loss = 4.521
Epoch   9 Batch    7/107   train_loss = 4.421
Epoch   9 Batch   12/107   train_loss = 4.345
Epoch   9 Batch   17/107   train_loss = 4.665
Epoch   9 Batch   22/107   train_loss = 4.597
Epoch   9 Batch   27/107   train_loss = 4.503
Epoch   9 Batch   32/107   train_loss = 4.366
Epoch   9 Batch   37/107   train_l

Epoch  16 Batch   78/107   train_loss = 3.920
Epoch  16 Batch   83/107   train_loss = 3.755
Epoch  16 Batch   88/107   train_loss = 4.107
Epoch  16 Batch   93/107   train_loss = 4.054
Epoch  16 Batch   98/107   train_loss = 3.943
Epoch  16 Batch  103/107   train_loss = 3.992
Epoch  17 Batch    1/107   train_loss = 4.045
Epoch  17 Batch    6/107   train_loss = 3.854
Epoch  17 Batch   11/107   train_loss = 3.977
Epoch  17 Batch   16/107   train_loss = 4.028
Epoch  17 Batch   21/107   train_loss = 3.918
Epoch  17 Batch   26/107   train_loss = 4.131
Epoch  17 Batch   31/107   train_loss = 4.043
Epoch  17 Batch   36/107   train_loss = 3.855
Epoch  17 Batch   41/107   train_loss = 3.876
Epoch  17 Batch   46/107   train_loss = 3.951
Epoch  17 Batch   51/107   train_loss = 3.923
Epoch  17 Batch   56/107   train_loss = 3.819
Epoch  17 Batch   61/107   train_loss = 3.961
Epoch  17 Batch   66/107   train_loss = 3.859
Epoch  17 Batch   71/107   train_loss = 4.046
Epoch  17 Batch   76/107   train_l

Epoch  25 Batch   10/107   train_loss = 3.333
Epoch  25 Batch   15/107   train_loss = 3.654
Epoch  25 Batch   20/107   train_loss = 3.555
Epoch  25 Batch   25/107   train_loss = 3.485
Epoch  25 Batch   30/107   train_loss = 3.419
Epoch  25 Batch   35/107   train_loss = 3.533
Epoch  25 Batch   40/107   train_loss = 3.562
Epoch  25 Batch   45/107   train_loss = 3.526
Epoch  25 Batch   50/107   train_loss = 3.444
Epoch  25 Batch   55/107   train_loss = 3.566
Epoch  25 Batch   60/107   train_loss = 3.528
Epoch  25 Batch   65/107   train_loss = 3.402
Epoch  25 Batch   70/107   train_loss = 3.470
Epoch  25 Batch   75/107   train_loss = 3.312
Epoch  25 Batch   80/107   train_loss = 3.364
Epoch  25 Batch   85/107   train_loss = 3.367
Epoch  25 Batch   90/107   train_loss = 3.517
Epoch  25 Batch   95/107   train_loss = 3.420
Epoch  25 Batch  100/107   train_loss = 3.532
Epoch  25 Batch  105/107   train_loss = 3.425
Epoch  26 Batch    3/107   train_loss = 3.482
Epoch  26 Batch    8/107   train_l

Epoch  33 Batch   49/107   train_loss = 3.226
Epoch  33 Batch   54/107   train_loss = 3.258
Epoch  33 Batch   59/107   train_loss = 3.245
Epoch  33 Batch   64/107   train_loss = 3.214
Epoch  33 Batch   69/107   train_loss = 3.161
Epoch  33 Batch   74/107   train_loss = 3.104
Epoch  33 Batch   79/107   train_loss = 3.208
Epoch  33 Batch   84/107   train_loss = 3.211
Epoch  33 Batch   89/107   train_loss = 3.315
Epoch  33 Batch   94/107   train_loss = 3.198
Epoch  33 Batch   99/107   train_loss = 3.217
Epoch  33 Batch  104/107   train_loss = 3.100
Epoch  34 Batch    2/107   train_loss = 3.143
Epoch  34 Batch    7/107   train_loss = 3.076
Epoch  34 Batch   12/107   train_loss = 3.182
Epoch  34 Batch   17/107   train_loss = 3.313
Epoch  34 Batch   22/107   train_loss = 3.246
Epoch  34 Batch   27/107   train_loss = 3.194
Epoch  34 Batch   32/107   train_loss = 2.962
Epoch  34 Batch   37/107   train_loss = 3.151
Epoch  34 Batch   42/107   train_loss = 3.167
Epoch  34 Batch   47/107   train_l

...

Epoch 158 Batch   99/107   train_loss = 1.709
Epoch 158 Batch  104/107   train_loss = 1.546
Epoch 159 Batch    2/107   train_loss = 1.590
Epoch 159 Batch    7/107   train_loss = 1.678
Epoch 159 Batch   12/107   train_loss = 1.612
Epoch 159 Batch   17/107   train_loss = 1.724
Epoch 159 Batch   22/107   train_loss = 1.722
Epoch 159 Batch   27/107   train_loss = 1.762
Epoch 159 Batch   32/107   train_loss = 1.595
Epoch 159 Batch   37/107   train_loss = 1.614
Epoch 159 Batch   42/107   train_loss = 1.714
Epoch 159 Batch   47/107   train_loss = 1.661
Epoch 159 Batch   52/107   train_loss = 1.729
Epoch 159 Batch   57/107   train_loss = 1.636
Epoch 159 Batch   62/107   train_loss = 1.701
Epoch 159 Batch   67/107   train_loss = 1.598
Epoch 159 Batch   72/107   train_loss = 1.773
Epoch 159 Batch   77/107   train_loss = 1.679
Epoch 159 Batch   82/107   train_loss = 1.705
Epoch 159 Batch   87/107   train_loss = 1.694
Epoch 159 Batch   92/107   train_loss = 1.616
Epoch 159 Batch   97/107   train_l

Epoch 167 Batch   31/107   train_loss = 1.729
Epoch 167 Batch   36/107   train_loss = 1.683
Epoch 167 Batch   41/107   train_loss = 1.553
Epoch 167 Batch   46/107   train_loss = 1.626
Epoch 167 Batch   51/107   train_loss = 1.651
Epoch 167 Batch   56/107   train_loss = 1.680
Epoch 167 Batch   61/107   train_loss = 1.609
Epoch 167 Batch   66/107   train_loss = 1.663
Epoch 167 Batch   71/107   train_loss = 1.651
Epoch 167 Batch   76/107   train_loss = 1.608
Epoch 167 Batch   81/107   train_loss = 1.621
Epoch 167 Batch   86/107   train_loss = 1.716
Epoch 167 Batch   91/107   train_loss = 1.606
Epoch 167 Batch   96/107   train_loss = 1.566
Epoch 167 Batch  101/107   train_loss = 1.671
Epoch 167 Batch  106/107   train_loss = 1.560
Epoch 168 Batch    4/107   train_loss = 1.638
Epoch 168 Batch    9/107   train_loss = 1.672
Epoch 168 Batch   14/107   train_loss = 1.581
Epoch 168 Batch   19/107   train_loss = 1.667
Epoch 168 Batch   24/107   train_loss = 1.619
Epoch 168 Batch   29/107   train_l

Epoch 175 Batch   70/107   train_loss = 1.607
Epoch 175 Batch   75/107   train_loss = 1.475
Epoch 175 Batch   80/107   train_loss = 1.598
Epoch 175 Batch   85/107   train_loss = 1.643
Epoch 175 Batch   90/107   train_loss = 1.597
Epoch 175 Batch   95/107   train_loss = 1.549
Epoch 175 Batch  100/107   train_loss = 1.573
Epoch 175 Batch  105/107   train_loss = 1.658
Epoch 176 Batch    3/107   train_loss = 1.623
Epoch 176 Batch    8/107   train_loss = 1.621
Epoch 176 Batch   13/107   train_loss = 1.661
Epoch 176 Batch   18/107   train_loss = 1.671
Epoch 176 Batch   23/107   train_loss = 1.570
Epoch 176 Batch   28/107   train_loss = 1.669
Epoch 176 Batch   33/107   train_loss = 1.565
Epoch 176 Batch   38/107   train_loss = 1.541
Epoch 176 Batch   43/107   train_loss = 1.625
Epoch 176 Batch   48/107   train_loss = 1.700
Epoch 176 Batch   53/107   train_loss = 1.612
Epoch 176 Batch   58/107   train_loss = 1.534
Epoch 176 Batch   63/107   train_loss = 1.571
Epoch 176 Batch   68/107   train_l

Epoch 184 Batch    2/107   train_loss = 1.511
Epoch 184 Batch    7/107   train_loss = 1.571
Epoch 184 Batch   12/107   train_loss = 1.504
Epoch 184 Batch   17/107   train_loss = 1.593
Epoch 184 Batch   22/107   train_loss = 1.678
Epoch 184 Batch   27/107   train_loss = 1.587
Epoch 184 Batch   32/107   train_loss = 1.532
Epoch 184 Batch   37/107   train_loss = 1.629
Epoch 184 Batch   42/107   train_loss = 1.646
Epoch 184 Batch   47/107   train_loss = 1.619
Epoch 184 Batch   52/107   train_loss = 1.600
Epoch 184 Batch   57/107   train_loss = 1.540
Epoch 184 Batch   62/107   train_loss = 1.482
Epoch 184 Batch   67/107   train_loss = 1.576
Epoch 184 Batch   72/107   train_loss = 1.617
Epoch 184 Batch   77/107   train_loss = 1.566
Epoch 184 Batch   82/107   train_loss = 1.645
Epoch 184 Batch   87/107   train_loss = 1.676
Epoch 184 Batch   92/107   train_loss = 1.561
Epoch 184 Batch   97/107   train_loss = 1.608
Epoch 184 Batch  102/107   train_loss = 1.643
Epoch 185 Batch    0/107   train_l

Epoch 192 Batch   41/107   train_loss = 1.567
Epoch 192 Batch   46/107   train_loss = 1.567
Epoch 192 Batch   51/107   train_loss = 1.513
Epoch 192 Batch   56/107   train_loss = 1.573
Epoch 192 Batch   61/107   train_loss = 1.537
Epoch 192 Batch   66/107   train_loss = 1.562
Epoch 192 Batch   71/107   train_loss = 1.559
Epoch 192 Batch   76/107   train_loss = 1.474
Epoch 192 Batch   81/107   train_loss = 1.416
Epoch 192 Batch   86/107   train_loss = 1.592
Epoch 192 Batch   91/107   train_loss = 1.473
Epoch 192 Batch   96/107   train_loss = 1.477
Epoch 192 Batch  101/107   train_loss = 1.525
Epoch 192 Batch  106/107   train_loss = 1.485
Epoch 193 Batch    4/107   train_loss = 1.520
Epoch 193 Batch    9/107   train_loss = 1.551
Epoch 193 Batch   14/107   train_loss = 1.427
Epoch 193 Batch   19/107   train_loss = 1.583
Epoch 193 Batch   24/107   train_loss = 1.518
Epoch 193 Batch   29/107   train_loss = 1.643
Epoch 193 Batch   34/107   train_loss = 1.498
Epoch 193 Batch   39/107   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [19]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    #9 TODO: Implement Function

    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    initial_state_tensor = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state_tensor = loaded_graph.get_tensor_by_name("final_state:0")
    probs_tensor = loaded_graph.get_tensor_by_name("probs:0")
    return input_tensor, initial_state_tensor, final_state_tensor, probs_tensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [20]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    #10 TODO: Implement Function

    return int_to_vocab[np.random.choice(np.arange(len(int_to_vocab)), p=probabilities)]

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [21]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak: aw, that's the barbed thing. how we have you workin' any eggs and you're a pig. i know it says just just like a nice what help?
marge_simpson: maybe it are, ah, join up on trouble.
moe_szyslak: is that stuck up.


homer_simpson: you know, i never not to come around. renee? the springfield top off, a ball) all right, andalay!(more sobs)
barney_gumble: i don't know. everything would have kinda a little for" that you gave me something for some? what happened?
lisa_simpson:(on moe,) which is your son, here. 'cause i think. the rest of speech. you've can get to dr.
moe_szyslak:(conspiratorial) nigeria know here, sir. who don't want you there.
moe_szyslak: listen this.
homer_simpson: moe, yeah, moe, ah, bart. i gotta get back to the little woman.(beat) or this is the best thing to come up.
moe_szyslak: no dice. a


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.